In [1]:
### Imports from algo_common

import csv
import datetime
from dateutil import tz
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
import numpy as np
import os
import pandas as pd
import pytz
import sys
import time

os.chdir(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy')
sys.path.append(os.path.join(os.getcwd(), '..'))

from trader_factory import build_active_IBridgePy_plus
from IBridgePy.IbridgepyTools import symbol

from Utils import Algos
from Utils import EnterExits
from Utils import IndicatorCalculations
from Utils import Orders
from Utils import Plotters
from Utils import Utils
from UnitTests import testutils


In [ ]:
# All tests with orders are going to have to be tested using the testMe feature
# Tests: 
# test_checklong             - done
# test_checkShort            - done
# test_checkEnter            - done
# test_ checkSellLong        - done
# test_checkSellShort        - done
# test_checkExit-------------------
# test_checkCross            - done
# test_checkDoubleOrNothing--------
# test_generateBuyOrder------------
# test_generateSellOrder-----------

In [ ]:
# Input Data Needed

# context
# ema generator
# price generator


In [ ]:
def test_checklong():
""" four testing conditions:
    1. Price>fast>slow and fast<fast1: False
    2. Price>fast>slow and fast>fast1: True
    3. Price>fast<slow and fast<fast1: False
    4. Price>fast<slow and fast>fast1: False """

    knownOutcomes = [False, True, False, False] # vector containing outcomes
    outcomes = [] # empty list for appending outcomes

    # intialize all inputs
    price = np.ones(4)*100
    fast = np.ones(4)*99
    slow = np.array([98, 98, 101, 101])
    fast1 = np.array([100, 97, 100, 97])
    macd = np.ones(4)*100
    signalLine = np.array([100,99,100,100])
    
    # intialize context class
    context = testutils.contextNoSim()
    
    # loop through and calcualte checkLong
    for i in range(4):
        emaVec = np.array([fast[i], fast1[i], slow[i], slow[i], 0, 0, macd[i], macd[i], signalLine[i] ])
        priceVec = testutils.Price(price[i],price[i],price[i])
        ema = testutils.emaTestData(emaVec)
        outcomes.append(EnterExits.checkLong(context, ema, priceVec))

    assert np.all([outcomes[i]==knownOutcomes[i] for i in range(len(outcomes))]), "Results do not match"

In [ ]:
def test_checkShort():
    """ four testing conditions:
        Price<fast<slow and fast<fast1: True
        Price<fast>slow and fast>fast1: False
        Price>fast<slow and fast<fast1: False
        Price>fast<slow and fast>fast1: False """

    knownOutcomes = [True, False, False, False] # vector containing outcomes
    outcomes = [] # empty list for appending outcomes

    # intialize all inputs
    price = np.ones(4)*97
    fast = np.array([98, 98, 96, 96])
    slow = np.array([99, 97, 100, 101])
    fast1 = np.array([100, 97, 92, 95])
    macd = np.ones(4)*100
    signalLine = np.array([101,99,100,100])

    # intialize context class
    context = testutils.contextNoSim()

    # loop through and calcualte checkShort
    for i in range(4):
        emaVec = np.array([fast[i], fast1[i], slow[i], slow[i], 0, 0, macd[i], macd[i], signalLine[i] ])
        priceVec = testutils.Price(price[i],price[i],price[i])
        ema = testutils.emaTestData(emaVec)
        outcomes.append(EnterExits.checkShort(context, ema, priceVec))

    assert np.all([outcomes[i]==knownOutcomes[i] for i in range(len(outcomes))]), "Results do not match"



In [30]:
def test_checkEnter():
    """ This test has the following conditions:
    Price<fast<slow and fast<fast1: checkShort is True
    Price>fast>slow and fast>fast1: checkLong is True
    Price>fast<slow and fast<fast1: False
    Price>fast>slow and fast<fast1: False
    """
    
    # vectors containing outcomes
    knownOutcomes     = [True, True, False, False] 
    longBoolOutcomes  = [False, True, False, False]
    shortBoolOutcomes = [True, False, False, False]

    # empty lists for appending outcomes
    outcomes =   [] 
    longBools =  []
    shortBools = []

    # intialize all inputs
    price = np.ones(4)*98
    fast = np.array([99, 97, 96, 96])
    slow = np.array([100, 96, 100, 95])
    fast1 = np.array([100, 96, 97, 99])
    slow1 = np.array([97, 97, 96, 101])
    macd = np.ones(4)*100
    signalLine = np.array([101,98,100,100])

    # loop through and calcualte checkCross
    for i in range(4):
        # Get ema class
        emaVec = np.array([fast[i], fast1[i], slow[i], slow1[i], 0, 0, macd[i], macd[i], signalLine[i]])
        ema = testutils.emaTestData(emaVec)

        context = testutils.contextNoSim() # intialize/reset context class
        priceVec = testutils.Price(price[i],price[i],price[i]) # get price class

        # check to enter
        EnterExits.checkEnter(context, ema, priceVec)

        # record flags
        outcomes.append(context.enter_flag)
        longBools.append(context.longBool)
        shortBools.append(context.shortBool)

    assert np.all([outcomes[i]==knownOutcomes[i] for i in range(len(outcomes))]), "Results do not match"
    assert np.all([shortBools[i]==shortBoolOutcomes[i] for i in range(len(outcomes))]), "Shorts do not match"
    assert np.all([longBools[i]==longBoolOutcomes[i] for i in range(len(outcomes))]), "Longs do not match"

test_checkEnter()



checking short

Checking short before the chop

We passed the chop, buy short
checking long

checking short

checking long

Checking Long before the chop

We passed the chop, buy long
checking short

checking long

checking short

checking long

Checking Long before the chop



In [7]:
def test_checkSellLong():
    """ Two testing conditions
    price<fast: True
    price>fast: False
    """
    
    knownOutcomes = [True, False] # vector containing outcomes
    outcomes = [] # empty list for appending outcomes

    # intialize all inputs
    price = np.ones(2)*97
    fast = np.array([98, 96])
    slow = np.array([99, 97])
    fast1 = np.array([100, 97])
    macd = np.ones(2)*100
    signalLine = np.array([101,99])

    # intialize context class
    context = testutils.contextNoSim()

    # loop through and calcualte checkSellLong
    for i in range(2):
        emaVec = np.array([fast[i], fast1[i], slow[i], slow[i], 0, 0, macd[i], macd[i], signalLine[i] ])
        priceVec = testutils.Price(price[i],price[i],price[i])
        ema = testutils.emaTestData(emaVec)
        outcomes.append(EnterExits.checkSellLong(context, ema, priceVec))

        assert np.all([outcomes[i]==knownOutcomes[i] for i in range(len(outcomes))]), "Results do not match"



checking to sell long 

Exit long Position
checking to sell long 



In [10]:
def test_checkSellShort():
    """ Two testing conditions
    price>fast: True
    price<fast: False
    """
    
    knownOutcomes = [True, False] # vector containing outcomes
    outcomes = [] # empty list for appending outcomes

    # intialize all inputs
    price = np.ones(2)*97
    fast = np.array([96, 98])
    slow = np.array([99, 97])
    fast1 = np.array([100, 97])
    macd = np.ones(2)*100
    signalLine = np.array([101,99])

    # intialize context class
    context = testutils.contextNoSim()

    # loop through and calcualte checkSellShort
    for i in range(2):
        emaVec = np.array([fast[i], fast1[i], slow[i], slow[i], 0, 0, macd[i], macd[i], signalLine[i] ])
        priceVec = testutils.Price(price[i],price[i],price[i])
        ema = testutils.emaTestData(emaVec)
        outcomes.append(EnterExits.checkSellShort(context, ema, priceVec))

        assert np.all([outcomes[i]==knownOutcomes[i] for i in range(len(outcomes))]), "Results do not match"


In [22]:
def test_checkCross():
    """ 4 testing conditions:
    fast>slow and fast1<slow1: True - long
    fast>slow and fast1>slow1: False
    fast<slow and fast1>slow1: True - short
    fast<slow and fast1<slow1: False
    """
    
    # vectors containing outcomes
    knownOutcomes     = [True, False, True, False] 
    longBoolOutcomes  = [True, False, False, False]
    shortBoolOutcomes = [False, False, True, False]

    # empty lists for appending outcomes
    outcomes =   [] 
    longBools =  []
    shortBools = []

    # intialize all inputs
    fast = np.array([99, 99, 96, 96])
    slow = np.array([96, 97, 100, 101])
    fast1 = np.array([96, 98, 97, 95])
    slow1 = np.array([97, 97, 96, 101])
    macd = np.ones(4)*100
    signalLine = np.array([101,99,100,100])

    # loop through and calcualte checkCross
    for i in range(4):
        context = testutils.contextNoSim() # intialize/reset context class
        emaVec = np.array([fast[i], fast1[i], slow[i], slow1[i], 0, 0, macd[i], macd[i], signalLine[i]])
        ema = testutils.emaTestData(emaVec)
        outcomes.append(EnterExits.checkCross(context, ema))
        longBools.append(context.longBool)
        shortBools.append(context.shortBool)

    assert np.all([outcomes[i]==knownOutcomes[i] for i in range(len(outcomes))]), "Results do not match"
    assert np.all([shortBools[i]==shortBoolOutcomes[i] for i in range(len(outcomes))]), "Shorts do not match"
    assert np.all([longBools[i]==longBoolOutcomes[i] for i in range(len(outcomes))]), "Longs do not match"

test_checkCross()

In [ ]:
def test_doubleOrNothing():
    """ 4 test conditions: 
    Long: 
        Tline>slow and fast>slow and macd>signalLine: buy long
        Tline<slow and fast>slow and macd>signalLine: exit
    Short: 
        Tline<slow and fast<slow and macd<signalLine: buy short
        Tline<slow and fast>slow and macd>signalLine: exit
    """

In [ ]:
def test_checkExit():
    """ 4 test conditions:
    - check cross occur exit works
          - checkSellShort is True, shortBool is True
          - Timer>60
          - Cross Occurs
    - check all 3 double or nothing conditions
     - exit position:
          - checkSellShort is True, shortBool is True
          - Timer>60
          - Cross doesnt occur
          - Tline<slow and fast<slow and macd>signalLine
     - double long:
          - checkSellLong is True, longBool is True
          - Timer>60
          - Cross doesnt occur
          - Tline>slow and fast>slow and macd>signalLine
     - double short
          - checkSellShort is True, shortBool is True
          - Timer>60
          - Cross doesnt occur
          - Tline<slow and fast<slow and macd<signalLine """


In [ ]:
#def test_generateBuyOrder():

In [ ]:
# def test_generateSellOrder():


In [31]:
# # def test_getFillStatus():
#     """ 4 test conditions: 
#     Get real time price and:
#     Long:
#         - set ask price >>> real price: Tue
#         - set ask price <<< real price: False
#     Short:
#         - set ask price >>> real price: False
#         - set ask price <<< real price: True"""
    
#     knownOutcomes = [False, True, True, False]
#     outcomes = []
    
#     ############# Try to buy long for more than its worth - True #################
#     i=0 
#     priceReal = Utils.Price(context,data) # get price
#     askPrice = testutils.Price(priceReal.ask/10,priceReal.ask/10,priceReal.ask/10) # create price

#     if i==0:
#         context.exit_order= order(context.security, 10, LimitOrder(np.round(askPrice.mid,2)), accountCode='default')
    
#     if i>5:
#         outcomes.append(Orders.checkFillstatus(context,data))
#     i+=1
    
#     ############# Try to buy long for less than its worth - False #################
#     j=0
#     priceReal = Utils.Price(context,data) # get price
#     askPrice = testutils.Price(priceReal.ask/10,priceReal.ask/10,priceReal.ask/10) # create price

#     if j==0:
#         context.exit_order= order(context.security, 10, LimitOrder(np.round(askPrice.mid,2)), accountCode='default')
    
#     if j>5:
#         outcomes.append(Orders.checkFillstatus(context,data))
#     j+=1
    
#     ############# Try to buy short for more than its worth - False #################
#     k=0
#     priceReal = Utils.Price(context,data) # get price
#     askPrice = testutils.Price(priceReal.ask/10,priceReal.ask/10,priceReal.ask/10) # create price

#     if k==0:
#         context.exit_order= order(context.security, 10, LimitOrder(np.round(askPrice.mid,2)), accountCode='default')
    
#     if k>5:
#         outcomes.append(Orders.checkFillstatus(context,data))
#     k+=1
    
#     ############# Try to buy short for less than its worth - True #################
#     l=0
#     priceReal = Utils.Price(context,data) # get price
#     askPrice = testutils.Price(priceReal.ask/10,priceReal.ask/10,priceReal.ask/10) # create price

#     if l==0:
#         context.exit_order= order(context.security, 10, LimitOrder(np.round(askPrice.mid,2)), accountCode='default')
    
#     if l>5:
#         outcomes.append(Orders.checkFillstatus(context,data))
#     l+=1

In [32]:
def test_getFillStatus():
    """ 4 test conditions: 
    Get real time price and:
    Long:
        - set ask price >>> real price: Tue
        - set ask price <<< real price: False
    Short:
        - set ask price >>> real price: False
        - set ask price <<< real price: True"""
    
    knownOutcomes = [True, False, True, False]
    outcomes = []
    
    ############# Try to buy long for more than its worth - True #################
    i=0 
    priceReal = Utils.Price(context,data) # get price
    askPriceLow = testutils.Price(priceReal.ask/10,priceReal.ask/10,priceReal.ask/10) # create price
    askPriceHigh = testutils.Price(priceReal.ask*10,priceReal.ask*10,priceReal.ask*10) # create price
    for i in range(5): # buy long at high Price
        if i==0:
            context.exit_order= order(context.security, 10, LimitOrder(np.round(askPriceHigh.mid,2)), accountCode='default')
    outcomes.append(Orders.checkFillstatus(context,data))
    for i in range(5): # buy long at low price
        if i==0:
            context.exit_order= order(context.security, 10, LimitOrder(np.round(askPriceLow.mid,2)), accountCode='default')
    outcomes.append(Orders.checkFillstatus(context,data))
    for i in range(5): # buy short at high price
        if i==0:
            context.exit_order= order(context.security, -10, LimitOrder(np.round(askPriceHigh.mid,2)), accountCode='default')
    outcomes.append(Orders.checkFillstatus(context,data))
    for i in range(5): # buy short at low price
        if i==0:
            context.exit_order= order(context.security, -10, LimitOrder(np.round(askPriceLow.mid,2)), accountCode='default')
    outcomes.append(Orders.checkFillstatus(context,data))
    
print(outcomes)

[True, True, False, False]
